## Import libs

In [ ]:
%cd ..

import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

In [ ]:
!pip uninstall utils

In [ ]:
!pip install datasets
!pip install transformers
# !pip install utils.prepare_dataset

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, GPT2LMHeadModel
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import torch
#from progressbar import progressbar
from tqdm.auto import tqdm
from prepare_dataset import load_datasets, cut_datasets
from collections import defaultdict
import pickle
import psutil
import pandas as pd
import os
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

/Users/sashafedorova/jetBrainsIntern/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Define hyperparams

In [2]:
# Define datasets
#['mrpc', 'sst2', 'cola', 'rte', 'qnli']
glue_classification = {'mrpc': ['sentence1', 'sentence2']} #, 'sst2':  ['sentence']}
superglue_classification = {'wic': ['sentence1', 'sentence2']}
all_classification = {'glue': glue_classification, 'superglue': superglue_classification}

model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=1024)
model = AutoModel.from_pretrained(model_name)
model.eval()

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

os.environ["TOKENIZERS_PARALLELISM"] = "false"
model.to(device)
DEBUG_FLAG = False
CUT_SIZE = 1000 if not DEBUG_FLAG else 100

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Load datasets

In [3]:
glue_datasets = load_datasets('glue', list(glue_classification), CUT_SIZE)
superglue_datasets = load_datasets('super_glue', list(superglue_classification), CUT_SIZE)

all_datasets = {'glue': glue_datasets, 'superglue': superglue_datasets}

Using the latest cached version of the module from /Users/sashafedorova/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Tue Jun 20 04:32:12 2023) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
Found cached dataset glue (/Users/sashafedorova/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Using the latest cached version of the module from /Users/sashafedorova/.cache/huggingface/modules/datasets_modules/datasets/super_glue/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed (last modified on Tue Jun 20 04:52:35 2023) since it couldn't be found locally at super_glue., or remotely on the Hugging Face Hub.
Found cached dataset super_glue (/Users/sashafedorova/.cache/huggingface/datasets/super_glue/wic/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
list(glue_classification), list(superglue_classification)

In [ ]:
ex = glue_datasets['mrpc']['train'][0]
encoded_inputs = tokenizer.encode(
                    ex['sentence1'],
                    ex['sentence2'],
                    truncation=True,
                    return_tensors='pt'
                ).to(device)
aa = model(encoded_inputs, output_hidden_states=True, output_attentions=True)
i = 0
for t in aa.hidden_states:
    i+=1
    # print(t.size())
model.config.num_hidden_layers

In [ ]:
encoded_inputs = tokenizer.encode(
                    ex['sentence1'],
                    ex['sentence2'],
                    truncation=True,
                    return_tensors='pt'
                ).to(device)

## Collect dataset for layers and heads

In [4]:
def add_to_pickle_file(file_path, new_results):
    # Load existing data from the file
    try:
        with open(file_path, 'rb') as file:
            existing_data = pickle.load(file)
    except FileNotFoundError:
      existing_data = {}
    except EOFError:
      existing_data = {}

    for l, heads in new_results.items():
        if l not in existing_data:
            existing_data[l] = heads
            continue
        for h, data in heads.items():
            if h not in existing_data[l]:
                existing_data[l][h] = data
            else:
                print(existing_data[l][h])
                print(existing_data[l][h].update(data))
                print(existing_data)
                existing_data[l][h].update(data)

    with open(file_path, 'wb') as file:
        pickle.dump(existing_data, file)

def get_from_pickle_File(file_path):
    try:
        with open(file_path, 'rb') as file:
            return pickle.load(file)
    except FileNotFoundError:
        return {}
    except EOFError:
        return {}

name = 'try.pickle'
add_to_pickle_file(name, {1:{3:{2:0}}, 3:{}})
get_from_pickle_File(name)

{3: 0, 2: 0}
None
{1: {3: {3: 0, 2: 0}}, 3: {}}


{1: {3: {3: 0, 2: 0}}, 3: {}}

In [9]:
import numpy as np


layer2head2sample = {}
# sample = ((cur_emb, cur_pos, cur_len, tgt_pos, tgt_len), attention_weight)

for ex in tqdm(glue_datasets['mrpc']['train']):
    encoded_inputs = tokenizer.encode(
                    ex['sentence1'],
                    ex['sentence2'],
                    truncation=True,
                    return_tensors='pt'
                ).to(device)
    aa = model(encoded_inputs, output_hidden_states=True, output_attentions=True)

    for layer in [9]:
        if layer not in layer2head2sample:
            layer2head2sample[layer] = {}
        for head_num in range( aa.attentions[0].shape[1]): #
            if head_num not in layer2head2sample[layer]:
                layer2head2sample[layer][head_num] = []
            for from_ in range(aa.attentions[0].shape[2]):
                for to_ in range(aa.attentions[0].shape[3]):
                    cur_emb = aa.hidden_states[layer][0][from_].detach().cpu().numpy()
                    tgt_attention = aa.attentions[layer][0][head_num][from_][to_].detach().cpu().item()
                    layer2head2sample[layer][head_num].append(((cur_emb, from_, aa.attentions[0].shape[2], to_, aa.attentions[0].shape[3]), tgt_attention))


  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
# %%time
# for layer, heads in layer2head2sample.items():
#     for head in tqdm(heads):
#         torch.save(heads[head],f'data/layer{layer}/head{head}.pt', pickle_protocol = 5)

  0%|          | 0/11 [00:00<?, ?it/s]

RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 128 vs 0

In [7]:
poly_features = PolynomialFeatures(degree=degree)

# Transform the original feature matrix to include polynomial features
X_poly = poly_features.fit_transform(X)

In [12]:
results_new = {}
for layer, heads in layer2head2sample.items():
    if layer not in results_new:
        results_new[layer] = {}
    for head in tqdm(range(12)):
        if head not in results_new[layer]:
            results_new[layer][head] = {}
        samples = heads[head]
        samples_idxs = np.random.choice(range(len(samples)), size=200000, replace=False)
        X_train_relative_position = np.array([[samples[el][0][1]/samples[el][0][2], samples[el][0][1], samples[el][0][2] - samples[el][0][1], samples[el][0][2], samples[el][0][3]/samples[el][0][4], samples[el][0][4] - samples[el][0][3], samples[el][0][4]] for el in samples_idxs])
        y_train = np.array([samples[el][1] for el in samples_idxs])
        print(f'layer {layer} head {head} :')
        X_poly5 = PolynomialFeatures(degree=5).fit_transform(X_train_relative_position)
        # X_poly6 = PolynomialFeatures(degree=6).fit_transform(X_train_relative_position)
        # X_poly4 = PolynomialFeatures(degree=4).fit_transform(X_train_relative_position)
        print(f'hey')
        # len_emb = cross_val_score(Ridge(), X_train_relative_position, y_train, n_jobs=4, scoring='r2').mean()
        poly5 = cross_val_score(Ridge(), X_poly5, y_train, n_jobs=4, scoring='r2').mean()
        # poly6 = cross_val_score(Ridge(), X_poly6, y_train, n_jobs=4, scoring='r2').mean()
        # poly4 = cross_val_score(Ridge(), X_poly4, y_train, n_jobs=4, scoring='r2').mean()
        results_new[layer][head].update({'poly5': poly5})#, 'poly6': poly6})
        print(f'{results_new[layer][head]}')

  0%|          | 0/12 [00:00<?, ?it/s]

layer 9 head 0 :
hey
{'poly5': 0.15301797652495763}
layer 9 head 1 :
hey
{'poly5': 0.18676355863203983}
layer 9 head 2 :
hey
{'poly5': 0.157341799819982}
layer 9 head 3 :
hey
{'poly5': 0.23893186451635168}
layer 9 head 4 :
hey
{'poly5': 0.212714776007879}
layer 9 head 5 :
hey
{'poly5': 0.12567908338190836}
layer 9 head 6 :
hey
{'poly5': 0.18868649462245699}
layer 9 head 7 :
hey
{'poly5': 0.16941237992922825}
layer 9 head 8 :
hey
{'poly5': 0.0946308980106511}
layer 9 head 9 :
hey
{'poly5': 0.22708266326543125}
layer 9 head 10 :
hey
{'poly5': 0.1766131830258109}
layer 9 head 11 :
hey
{'poly5': 0.24510571871215642}


In [7]:
get_from_pickle_File(name)

{9: {0: {'base': 0.006259237851882782,
   'rev': 0.009365853995198403,
   'scaled': 0.00936561150954982,
   'pos': 0.006052847107969761,
   'len': 0.014863631025109703,
   'scaled_pos': 0.014863631235295549,
   'len_emb': 0.010657539535703853,
   'len_emb_scaled': 0.01065731162096133,
   'relative_position': 0.018115788464739,
   'poly2': 0.022308021681861433,
   'poly3': 0.05444942137401334,
   'poly4': 0.11771342110586316},
  1: {'base': 0.004311268172253135,
   'rev': 0.007082350977362406,
   'scaled': 0.00708209713143122,
   'pos': 0.004997279723827774,
   'len': 0.013200994460579808,
   'scaled_pos': 0.013200994819343559,
   'len_emb': 0.007747563561432557,
   'len_emb_scaled': 0.0077473157154580585,
   'relative_position': 0.015340255772600897,
   'poly2': 0.027117737627624615,
   'poly3': 0.05452475727552346,
   'poly4': 0.16334280512742577},
  2: {'base': 0.002884279699981729,
   'rev': 0.006928899023044765,
   'scaled': 0.0069286574317314106,
   'pos': 0.0023383856101595766,
 

In [51]:
results_new = {}
for layer in [9]:
    if layer not in results_new:
        results_new[layer] = {}
    for head, samples in layer2head2sample[layer].items():  #range(12):
        if head not in results_new[layer]:
            results_new[layer][head] = {}
        samples_idxs = np.random.choice(range(len(samples)), size=200000, replace=False)
        X_train_len_emb = np.array([list(samples[el][0][0])+[samples[el][0][1], samples[el][0][2] - samples[el][0][1], samples[el][0][2], samples[el][0][3], samples[el][0][4] - samples[el][0][3], samples[el][0][4]] for el in samples_idxs])
        y_train = np.array([samples[el][1] for el in samples_idxs])
        X_train_scaled = StandardScaler().fit_transform(X_train_len_emb)
        print(f'layer {layer} head {head} :')
        len_emb = cross_val_score(Ridge(), X_train_len_emb, y_train, n_jobs=4, scoring='r2').mean()
        len_emb_scaled = cross_val_score(Ridge(), X_train_scaled, y_train, n_jobs=4, scoring='r2').mean()
        results_new[layer][head].update({'len_emb': len_emb, 'len_emb_scaled': len_emb_scaled})
        print(f'{results_new[layer][head]}')

layer 9 head 0 :
{'len_emb': 0.010657539535703853, 'len_emb_scaled': 0.01065731162096133}
layer 9 head 1 :
{'len_emb': 0.007747563561432557, 'len_emb_scaled': 0.0077473157154580585}
layer 9 head 2 :
{'len_emb': 0.006788175926694539, 'len_emb_scaled': 0.006787934293901255}
layer 9 head 3 :
{'len_emb': 0.017982384732917932, 'len_emb_scaled': 0.01798213012173766}
layer 9 head 4 :
{'len_emb': 0.014002060918018788, 'len_emb_scaled': 0.01400180065150174}
layer 9 head 5 :
{'len_emb': 0.007321918680925399, 'len_emb_scaled': 0.007321706323502308}
layer 9 head 6 :
{'len_emb': 0.009881149229450515, 'len_emb_scaled': 0.009880894674065232}
layer 9 head 7 :
{'len_emb': 0.010926917919214319, 'len_emb_scaled': 0.010926689439320403}
layer 9 head 8 :
{'len_emb': 0.005379311924951047, 'len_emb_scaled': 0.005379042407306467}
layer 9 head 9 :
{'len_emb': 0.016474821864224352, 'len_emb_scaled': 0.016474562544888195}
layer 9 head 10 :
{'len_emb': 0.012390186592206875, 'len_emb_scaled': 0.01238992587799217}
l

In [47]:
results_new = {}
for layer, heads in layer2head2sample.items():
    if layer not in results_new:
        results_new[layer] = {}
    for head, samples in heads.items():  #range(12):
        if head not in results_new[layer]:
            results_new[layer][head] = {}
        samples_idxs = np.random.choice(range(len(samples)), size=200000, replace=False)

        X_train_base = np.array([list(samples[el][0][0]) + [samples[el][0][1], samples[el][0][3]] for el in samples_idxs])
        X_train_rev = np.array([list(samples[el][0][0]) + [samples[el][0][1], samples[el][0][2] - samples[el][0][1], samples[el][0][3], samples[el][0][4] - samples[el][0][3]] for el in samples_idxs])
        y_train = np.array([samples[el][1] for el in samples_idxs])
        X_train_scaled = StandardScaler().fit_transform(X_train_rev)

        base = cross_val_score(Ridge(), X_train_base, y_train, n_jobs=4, scoring='r2').mean()
        rev = cross_val_score(Ridge(), X_train_rev, y_train, n_jobs=4, scoring='r2').mean()
        scaled = cross_val_score(Ridge(), X_train_scaled, y_train, n_jobs=4, scoring='r2').mean()
        results_new[layer][head].update({'base': base, 'rev': rev, 'scaled': scaled})

        X_train_pos = np.array([[samples[el][0][1], samples[el][0][3]] for el in samples_idxs])
        X_train_len = np.array([[samples[el][0][1], samples[el][0][2] - samples[el][0][1],samples[el][0][2], samples[el][0][3], samples[el][0][4] - samples[el][0][3], samples[el][0][4]] for el in samples_idxs])
        X_train_scaled_pos = StandardScaler().fit_transform(X_train_len)
        print(f'layer {layer} head {head} :')
        pos = cross_val_score(Ridge(), X_train_pos, y_train, n_jobs=4, scoring='r2').mean()
        l = cross_val_score(Ridge(), X_train_len, y_train, n_jobs=4, scoring='r2').mean()
        scaled_pos = cross_val_score(Ridge(), X_train_scaled_pos, y_train, n_jobs=4, scoring='r2').mean()
        results_new[layer][head].update({'pos': pos, 'len': l, 'scaled_pos': scaled_pos})

        X_train_len_emb = np.array([list(samples[el][0][0])+[samples[el][0][1], samples[el][0][2] - samples[el][0][1], samples[el][0][2], samples[el][0][3], samples[el][0][4] - samples[el][0][3], samples[el][0][4]] for el in samples_idxs])
        X_train_scaled = StandardScaler().fit_transform(X_train_len_emb)

        len_emb = cross_val_score(Ridge(), X_train_len_emb, y_train, n_jobs=4, scoring='r2').mean()
        len_emb_scaled = cross_val_score(Ridge(), X_train_scaled, y_train, n_jobs=4, scoring='r2').mean()
        results_new[layer][head].update({'len_emb': len_emb, 'len_emb_scaled': len_emb_scaled})

        print(f'layer {layer} head {head} :')
        print(f'{results_new[layer][head]}')

layer 11 head 0 :
layer 11 head 0 :
{'base': 0.013378080594448071, 'rev': 0.020225867488554306, 'scaled': 0.02022557812249772, 'pos': 0.011987796716214217, 'len': 0.02524864074111701, 'scaled_pos': 0.025248641031803198, 'len_emb': 0.020225867487714357, 'len_emb_scaled': 0.020225577774493853}
layer 11 head 1 :
layer 11 head 1 :
{'base': 0.014722640955701815, 'rev': 0.020996626969501463, 'scaled': 0.020996359265890586, 'pos': 0.013009356798205474, 'len': 0.025734485343982816, 'scaled_pos': 0.0257344855872208, 'len_emb': 0.020996626967829512, 'len_emb_scaled': 0.020996358573655893}
layer 11 head 2 :
layer 11 head 2 :
{'base': 0.01150393734438151, 'rev': 0.020925534027493576, 'scaled': 0.020925292668412566, 'pos': 0.006042725215091171, 'len': 0.025285226379289493, 'scaled_pos': 0.025285226470029, 'len_emb': 0.020925534024059566, 'len_emb_scaled': 0.020925291246492072}
layer 11 head 3 :
layer 11 head 3 :
{'base': 0.01461739500976338, 'rev': 0.020796437223260654, 'scaled': 0.0207961660980256

In [13]:
name = 'results.pickle'
add_to_pickle_file(name, results_new)
get_from_pickle_File(name)

{'base': 0.006259237851882782, 'rev': 0.009365853995198403, 'scaled': 0.00936561150954982, 'pos': 0.006052847107969761, 'len': 0.014863631025109703, 'scaled_pos': 0.014863631235295549, 'len_emb': 0.010657539535703853, 'len_emb_scaled': 0.01065731162096133, 'relative_position': 0.018115788464739, 'poly2': 0.022308021681861433, 'poly3': 0.05444942137401334, 'poly4': 0.11771342110586316}
None
{9: {0: {'base': 0.006259237851882782, 'rev': 0.009365853995198403, 'scaled': 0.00936561150954982, 'pos': 0.006052847107969761, 'len': 0.014863631025109703, 'scaled_pos': 0.014863631235295549, 'len_emb': 0.010657539535703853, 'len_emb_scaled': 0.01065731162096133, 'relative_position': 0.018115788464739, 'poly2': 0.022308021681861433, 'poly3': 0.05444942137401334, 'poly4': 0.11771342110586316, 'poly5': 0.15301797652495763}, 1: {'base': 0.004311268172253135, 'rev': 0.007082350977362406, 'scaled': 0.00708209713143122, 'pos': 0.004997279723827774, 'len': 0.013200994460579808, 'scaled_pos': 0.013200994819

{9: {0: {'base': 0.006259237851882782,
   'rev': 0.009365853995198403,
   'scaled': 0.00936561150954982,
   'pos': 0.006052847107969761,
   'len': 0.014863631025109703,
   'scaled_pos': 0.014863631235295549,
   'len_emb': 0.010657539535703853,
   'len_emb_scaled': 0.01065731162096133,
   'relative_position': 0.018115788464739,
   'poly2': 0.022308021681861433,
   'poly3': 0.05444942137401334,
   'poly4': 0.11771342110586316,
   'poly5': 0.15301797652495763},
  1: {'base': 0.004311268172253135,
   'rev': 0.007082350977362406,
   'scaled': 0.00708209713143122,
   'pos': 0.004997279723827774,
   'len': 0.013200994460579808,
   'scaled_pos': 0.013200994819343559,
   'len_emb': 0.007747563561432557,
   'len_emb_scaled': 0.0077473157154580585,
   'relative_position': 0.015340255772600897,
   'poly2': 0.027117737627624615,
   'poly3': 0.05452475727552346,
   'poly4': 0.16334280512742577,
   'poly5': 0.18676355863203983},
  2: {'base': 0.002884279699981729,
   'rev': 0.006928899023044765,
   '

In [14]:
res = get_from_pickle_File('results.pickle')

In [15]:
res

{9: {0: {'base': 0.006259237851882782,
   'rev': 0.009365853995198403,
   'scaled': 0.00936561150954982,
   'pos': 0.006052847107969761,
   'len': 0.014863631025109703,
   'scaled_pos': 0.014863631235295549,
   'len_emb': 0.010657539535703853,
   'len_emb_scaled': 0.01065731162096133,
   'relative_position': 0.018115788464739,
   'poly2': 0.022308021681861433,
   'poly3': 0.05444942137401334,
   'poly4': 0.11771342110586316,
   'poly5': 0.15301797652495763},
  1: {'base': 0.004311268172253135,
   'rev': 0.007082350977362406,
   'scaled': 0.00708209713143122,
   'pos': 0.004997279723827774,
   'len': 0.013200994460579808,
   'scaled_pos': 0.013200994819343559,
   'len_emb': 0.007747563561432557,
   'len_emb_scaled': 0.0077473157154580585,
   'relative_position': 0.015340255772600897,
   'poly2': 0.027117737627624615,
   'poly3': 0.05452475727552346,
   'poly4': 0.16334280512742577,
   'poly5': 0.18676355863203983},
  2: {'base': 0.002884279699981729,
   'rev': 0.006928899023044765,
   '

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

for layer in [0]:
    for head in [0]: #range(12):
        with torch.serialization._open_file(f'data/layer{layer}/head{head}.pt', 'rb') as f:
            samples = torch.load(f, map_location=torch.device('cpu'))
        samples_idxs = np.random.choice(range(len(samples)), size=200000, replace=False)
        X_train = np.array([list(torch.log(torch.tensor(samples[el][0][0]))) + [samples[el][0][1], samples[el][0][2] - samples[el][0][1], samples[el][0][3], samples[el][0][4] - samples[el][0][3]] for el in samples_idxs])
        y_train = np.array([samples[el][1] for el in samples_idxs])
        print(f'layer {layer} head {head} :')
        print(X_train.shape, y_train.shape)
        print(cross_val_score(Ridge(), X_train, y_train, n_jobs=4, scoring='r2'))


In [90]:
results1

{0: {'base': 0.006259237851882782,
  'rev': 0.009365853995198403,
  'scaled': 0.00936561150954982,
  'pos': 0.006052847107969761,
  'len': 0.014863631025109703,
  'scaled_pos': 0.014863631235295549},
 1: {'base': 0.004311268172253135,
  'rev': 0.007082350977362406,
  'scaled': 0.00708209713143122,
  'pos': 0.004997279723827774,
  'len': 0.013200994460579808,
  'scaled_pos': 0.013200994819343559},
 2: {'base': 0.002884279699981729,
  'rev': 0.006928899023044765,
  'scaled': 0.0069286574317314106,
  'pos': 0.0023383856101595766,
  'len': 0.011297076403004081,
  'scaled_pos': 0.011297076541682393},
 3: {'base': 0.014647077306350887,
  'rev': 0.018894426186571955,
  'scaled': 0.018894172308179646,
  'pos': 0.01267280781327711,
  'len': 0.02367930096638766,
  'scaled_pos': 0.02367930122562545},
 4: {'base': 0.010655752780729433,
  'rev': 0.014333156397586855,
  'scaled': 0.01433289446538999,
  'pos': 0.010398203366072023,
  'len': 0.019831793227170814,
  'scaled_pos': 0.019831793348501936},

In [89]:
results1 = results2[9]

In [68]:
results2 = results1.drop('pos', )

In [75]:
for l, hs in results2.items():
    print(l, hs)
    # for h, data in hs.items():
    #     results2[l][h].update(results[l][h])
results2

9 {0: {'base': 0.006259237851882782, 'rev': 0.009365853995198403, 'scaled': 0.00936561150954982, 'pos': 0.006052847107969761, 'len': 0.014863631025109703, 'scaled_pos': 0.014863631235295549}, 1: {'base': 0.004311268172253135, 'rev': 0.007082350977362406, 'scaled': 0.00708209713143122, 'pos': 0.004997279723827774, 'len': 0.013200994460579808, 'scaled_pos': 0.013200994819343559}, 2: {'base': 0.002884279699981729, 'rev': 0.006928899023044765, 'scaled': 0.0069286574317314106, 'pos': 0.0023383856101595766, 'len': 0.011297076403004081, 'scaled_pos': 0.011297076541682393}, 3: {'base': 0.014647077306350887, 'rev': 0.018894426186571955, 'scaled': 0.018894172308179646, 'pos': 0.01267280781327711, 'len': 0.02367930096638766, 'scaled_pos': 0.02367930122562545}, 4: {'base': 0.010655752780729433, 'rev': 0.014333156397586855, 'scaled': 0.01433289446538999, 'pos': 0.010398203366072023, 'len': 0.019831793227170814, 'scaled_pos': 0.019831793348501936}, 5: {'base': 0.0032031344820485862, 'rev': 0.0074728

{9: {0: {'base': 0.006259237851882782,
   'rev': 0.009365853995198403,
   'scaled': 0.00936561150954982,
   'pos': 0.006052847107969761,
   'len': 0.014863631025109703,
   'scaled_pos': 0.014863631235295549},
  1: {'base': 0.004311268172253135,
   'rev': 0.007082350977362406,
   'scaled': 0.00708209713143122,
   'pos': 0.004997279723827774,
   'len': 0.013200994460579808,
   'scaled_pos': 0.013200994819343559},
  2: {'base': 0.002884279699981729,
   'rev': 0.006928899023044765,
   'scaled': 0.0069286574317314106,
   'pos': 0.0023383856101595766,
   'len': 0.011297076403004081,
   'scaled_pos': 0.011297076541682393},
  3: {'base': 0.014647077306350887,
   'rev': 0.018894426186571955,
   'scaled': 0.018894172308179646,
   'pos': 0.01267280781327711,
   'len': 0.02367930096638766,
   'scaled_pos': 0.02367930122562545},
  4: {'base': 0.010655752780729433,
   'rev': 0.014333156397586855,
   'scaled': 0.01433289446538999,
   'pos': 0.010398203366072023,
   'len': 0.019831793227170814,
   'sc

dict_items([(0, {'base': 0.006259237851882782, 'rev': 0.009365853995198403, 'scaled': 0.00936561150954982, 'pos': 0.006052847107969761, 'len': 0.014863631025109703, 'scaled_pos': 0.014863631235295549}), (1, {'base': 0.004311268172253135, 'rev': 0.007082350977362406, 'scaled': 0.00708209713143122, 'pos': 0.004997279723827774, 'len': 0.013200994460579808, 'scaled_pos': 0.013200994819343559}), (2, {'base': 0.002884279699981729, 'rev': 0.006928899023044765, 'scaled': 0.0069286574317314106, 'pos': 0.0023383856101595766, 'len': 0.011297076403004081, 'scaled_pos': 0.011297076541682393}), (3, {'base': 0.014647077306350887, 'rev': 0.018894426186571955, 'scaled': 0.018894172308179646, 'pos': 0.01267280781327711, 'len': 0.02367930096638766, 'scaled_pos': 0.02367930122562545}), (4, {'base': 0.010655752780729433, 'rev': 0.014333156397586855, 'scaled': 0.01433289446538999, 'pos': 0.010398203366072023, 'len': 0.019831793227170814, 'scaled_pos': 0.019831793348501936}), (5, {'base': 0.00320313448204858

In [ ]:
layer, head_num

In [ ]:
cross_val_score(Ridge(), (X_train[:, [-2]] - X_train[:, [-1]]).reshape(200000, -1), y_train, n_jobs=4, scoring='r2')

In [ ]:
cross_val_score(Ridge(), X_train_scaled[:, [-2, -1]], y_train, n_jobs=4, scoring='r2')

In [ ]:
tqdm_pbar = lambda x, y: tqdm(x, leave=True, position=0, total=len(x), desc=f'{y}')
def get_embeddings_for_dataset(dataset_name, dataset,
                                   feature_names, model, tokenizer,
                                   pbar_func=tqdm_pbar, device=device, CUT_SIZE=CUT_SIZE):
    collected_embeddings = defaultdict(list)

    for split, data in dataset.items():

        pbar = pbar_func(data, f"{split} {dataset_name}") if pbar_func is not None else data
        for example in pbar:
            # Encode the input sentences
            if len(feature_names) == 2:
                encoded_inputs = tokenizer.encode(
                    example[feature_names[0]],
                    example[feature_names[1]],
                    truncation=True,
                    return_tensors='pt'
                )
            else:
                encoded_inputs = tokenizer.encode(*list(map(lambda x: example[x] , feature_names)),
                                              truncation=True,
                                              return_tensors='pt')

            encoded_inputs = encoded_inputs.to(device)

            # Forward pass through the model
            with torch.no_grad():
                outputs = model(encoded_inputs)

            # Get the embedding of the [CLS] token
            cls_embedding = outputs.last_hidden_state[:, 0, :]

            # Append the [CLS] embedding to the list
            collected_embeddings[split].append(cls_embedding)

    return collected_embeddings

In [ ]:
def train_linear(X_train, y_train):
    classifier = LogisticRegression(solver='lbfgs', max_iter=3000)
    classifier.fit(X_train, y_train)
    return classifier

def evaluate_classifier(classifier, X, y=None):
    predictions = classifier.predict(X)
    return predictions

def get_metrics_report(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred,  average='weighted')
    #precision = precision_score(y_true, y_pred,  average='weighted')
    #recall = recall_score(y_true, y_pred,  average='weighted', zero_division='warn')
    #roc_auc = roc_auc_score(y_true, y_pred)
    print(classification_report(y_true, y_pred))
    print('Weighted F1', f1)
    print('Accuracy', accuracy)

In [ ]:
for dn, datasets in all_datasets.items():
    for dataset_name, dataset in datasets.items():
        print(f"{dn.upper()} / {dataset_name}\n")
        dataset_embeddings = get_cls_embeddings_for_dataset(
            dataset_name,
            dataset,
            all_classification[dn][dataset_name],
            model,
            tokenizer)

        train_dataset_embeddings = torch.cat(dataset_embeddings['train'], dim=0).cpu()
        valid_dataset_embeddings = torch.cat(dataset_embeddings['validation'], dim=0).cpu()
        test_dataset_embeddings = torch.cat(dataset_embeddings['test'], dim=0).cpu()

        classif = train_linear(train_dataset_embeddings, [el['label'] for el in dataset['train']])
        valid_preds = evaluate_classifier(classif, valid_dataset_embeddings)
        print('Validation evaluation:\n')
        get_metrics_report([el['label'] for el in dataset['validation']], valid_preds)
        # print(train_dataset_embeddings.shape)